In [2]:
import time
import re
import sys
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [14]:
url_uber = 'https://www.ubereats.com/mx/feed?ps=1&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMkNhdGVkcmFsJTIwTWV0cm9wb2xpdGFuYSUyMGRlJTIwbGElMjBDaXVkYWQlMjBkZSUyME0lQzMlQTl4aWNvJTIyJTJDJTIycmVmZXJlbmNlJTIyJTNBJTIyMzNjOWJiZGItNWY3OC01MGY2LTdmNmMtNDVmNzY4MzEzOTc5JTIyJTJDJTIycmVmZXJlbmNlVHlwZSUyMiUzQSUyMnViZXJfcGxhY2VzJTIyJTJDJTIybGF0aXR1ZGUlMjIlM0ExOS40MzAxNDQ2JTJDJTIybG9uZ2l0dWRlJTIyJTNBLTk5LjEzMzkxODYlN0Q%3D'

driver = webdriver.Chrome()

driver.get(url)

In [15]:
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
button = driver.find_element_by_xpath('//button[normalize-space()="Mostrar más"]')
ActionChains(driver).click(button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
button = driver.find_element_by_xpath('//button[normalize-space()="Mostrar más"]')
ActionChains(driver).click(button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()

page_source = driver.page_source

In [16]:
driver.quit()

Uber = BeautifulSoup(page_source, 'lxml')

In [17]:
print(Uber.prettify())

<html class="js-focus-visible" data-rh="translate,lang" dir="ltr" lang="es-MX" translate="no">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pide comida en línea | App para entrega de comida a domicilio | Uber Eats
  </title>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-main-5f1868d8d930d8eb4391.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-vendor-b535e1b2999f81ab9e86.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-default-cart-mobile-cart-mobile-multistep-cart-mobile-online-ordering-checkout-desktop-multistep-roo-6210a6ce-4e20e385ba60f437bb04.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-default-active-order-mobile-root-active-order-mobile-root-online-ordering

In [33]:
# store names = h3
# delivery fee = <span class="cb ex cq b3">
# rating = <div aria-hidden="true" class="gp cc cd ec eb hp cf gh ag c0 gd">


names = Uber.select('h3')
names = [name.text for name in names]
names

['Restaurante Zesta Punta',
 'Cónico',
 'Di Paulos (Centro)',
 'Dynasty (PORTAL CENTRO)',
 'Pirates Burgers (Isabel la Cátolica 28)',
 'Flying Burgers (Del Valle)',
 'Sushi Itto (Reforma 222)',
 'Pizzeria Cancino Alameda',
 'Sushi Itto Puerta Tlatelolco',
 '1/4 de Libra Burgers (Versalles)',
 'El Globo 5 de Febrero',
 'Krispy Kreme (Madero)',
 'Burger King (Madero)',
 'Little Caesars Metro insurgentes',
 "McDonald's Palma",
 'Starbucks (JUAREZ 32)',
 'Moyo (Madero)',
 'GOmart 🛒(Madero)',
 'Sushi Itto Puerta Tlatelolco',
 '1/4 de Libra Burgers (Versalles)',
 'El Globo 5 de Febrero',
 'Krispy Kreme (Madero)',
 'Burger King (Madero)',
 'Moshi Moshi Roma',
 'Little Caesars Metro insurgentes',
 'Sushi Itto (Reforma 222)',
 'Frutos Prohibidos y Otros Placeres Roma',
 'Subway (Centro Histórico CDMX)',
 'Grainz by Mora Mora Condesa',
 'Mora Mora Roma',
 'Green Grass Reforma',
 'Gracias Madre Taqueria Vegana',
 'Giornale Roma',
 'Eno Roma',
 'Carl´s Jr. Liverpool (Zona Rosa)',
 "Carl's Jr Indep

In [31]:
delivery_fee = Uber.find_all('div', class_='m9')
delivery_fee = [fee.text for fee in delivery_fee]
len(delivery_fee)

313

# Rappi

In [6]:
url_rappi = 'https://www.rappi.com.mx/restaurantes'

driver = webdriver.Chrome()

driver.get(url_rappi)

In [7]:
popup_button = driver.find_element_by_xpath('//*[@id="address-modal"]/div/div/div[1]/div[2]/div[2]/div/div/div[1]/div[1]/i')
ActionChains(driver).click(popup_button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
more_button = driver.find_element_by_xpath('//*[@id="app-container"]/div/span/app-by-stores-home/div[1]/div[2]/div[2]/button')
ActionChains(driver).click(more_button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()

page_source = driver.page_source

In [8]:
driver.quit()

In [9]:
rappi_soup = BeautifulSoup(page_source,'lxml')

In [12]:
print(rappi_soup.prettify())

<html lang="es">
 <head>
  <script async="" src="https://connect.facebook.net/signals/config/200737147691461?v=2.9.39&amp;r=stable">
  </script>
  <script async="" crossorigin="anonymous" src="https://connect.facebook.net/es_LA/sdk.js?hash=ea1fceb726c8779caf1566168ae0dab2&amp;ua=modern_es6">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/544894035678829?v=2.9.39&amp;r=stable">
  </script>
  <script async="" src="https://connect.facebook.net/en_US/fbevents.js">
  </script>
  <script async="" src="https://js.appboycdn.com/web-sdk/2.4/appboy.min.js" type="text/javascript">
  </script>
  <script async="" src="https://cdn.amplitude.com/libs/amplitude-4.2.1-min.gz.js" type="text/javascript">
  </script>
  <script async="" src="https://p.teads.tv/teads-fellow.js" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript">
  </script>
  <script async="" src="//www.googleadservices.com/pagead

In [13]:
rappi_stores = rappi_soup.select('h2', class_='store-name')

In [16]:
rappi_stores_names = [store.text for store in rappi_stores]
len(rappi_stores_names)

80

In [19]:
rappi_cuisines = rappi_soup.select('h3',class_='store-tags')
rappi_cuisines_names = [cuisine.text for cuisine in rappi_cuisines]
len(rappi_cuisines_names)

86

In [26]:
rappi_time = rappi_soup.find_all('span[class = "store-filters"] span[class = "store-eta ng-star-inserted"]')
rappi_delivery_time = [time.text for time in rappi_time]
rappi_delivery_time

[]